# **PyTorch: Internal Feature Map Calculation**

In [43]:
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
import torch
%matplotlib inline

In [4]:
# save pickle
def save_pickle(filename, obj):
    with open(str(filename), 'wb') as f:
        pickle.dump(obj, f)
    print("Object saved")
        
# load pickle
def load_pickle(filename):
    with open(str(filename), 'rb') as f:
        obj = pickle.load(f)
    print("Object loaded")
    return obj

In [5]:
import YoloDemonstration
model, device = YoloDemonstration.load_model('yolov7.pt')
pred, tensor_collection = YoloDemonstration.img_prediction('input/cat.jpg', model, device)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IMG SIZE = torch.Size([1, 3, 448, 640])


C:\Users\flfo\Anaconda3\envs\CNN_Vis\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
tensor_collection = load_pickle('tensor_collection.pkl')

Object loaded


In [8]:
test= tensor_collection[0][0][0].detach().clone()

In [32]:
test.size()

torch.Size([32, 448, 640])

In [35]:
test.size()[0]

32

In [38]:
test2 = test[0]

In [42]:
test2.size()

torch.Size([448, 640])

In [27]:
test[0][0][0]

tensor(-0.07476)

In [ ]:
torch.Tensor([255], )

In [93]:
def calc_grayscale(layer_map, norm_val):
    """
    Calculates normalized matrices in an array of 2D arrays and maps them to grayscale
    Input: NumPy array with 3 dimensions (num_pic, width, height)
    Output: modified NumPy array with same 3 dimensions
    """
    #start = time.time()
    #norm_val = torch.as_tensor([255], dtype=torch.float32)
    
    for i in range(layer_map.size()[0]):
        #img = test_map[i,:,:]
        img = layer_map[i]
        #print("TEST")
        v_min = img.min()
        v_max = img.max()
        img -= v_min
        img /= (v_max - v_min)
        img *= norm_val
   
    return layer_map


def calc_grayscale_dataset(data_dict, layer_nums=[]):
    """
    For whole data set as PyTorch Yolo tensor:
    Calculates normalized matrices in a list of 2D arrays and maps them to grayscale
    
    Input: User-defined tensor collection library added to the Yolo Model as attribute
    Output: Dictionary with layer number as key and corresponding normalized feature maps
    """
    
    ret_dict = {}
    norm_val = torch.as_tensor([255], dtype=torch.float32)
    
    if layer_nums:
        keys = layer_nums
    else:
        keys = data_dict.keys()

    for key in keys:
        # get PyTorch Yolo tensor with 3 dimensions (num_pics, width, height)
        # transform to NumPy array
        ret_dict[key] = calc_grayscale(data_dict[key][0][0].detach(), norm_val) 
    
    return ret_dict

In [94]:
test_collection = tensor_collection

In [96]:
#%%timeit
#layer_nums = [0,1,2,30,60,90,100,104]
layer_nums = []
s = time.time()
ret_dict = calc_grayscale_dataset(test_collection, layer_nums=layer_nums)
e = time.time()
print(f"Time consumed: {(e-s) * 1000} ms")

Time consumed: 1153.9371013641357 ms


In [68]:
ret_dict[0].size()

torch.Size([32, 448, 640])

In [1]:
import torch

In [2]:
torch.conv2d

<function torch._VariableFunctionsClass.conv2d>